In [1]:
import os
import h5py
import numpy as np
from matplotlib import image
import matplotlib.pyplot as plot
import numpy as np
import json
import io
import unicodecsv as csv
import tensorflow as tf
import gc
#import PIL
#from PIL import Image
#print('Pillow Version:', PIL.__version__)

import numpy as np
import tensorflow.keras as tkeras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Dropout
from tensorflow.keras.layers import Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, LSTM
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
#from keras.applications.imagenet_utils import preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras import optimizers



import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.utils import plot_model
#from resnets_utils import *
from tensorflow.keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import tensorflow.keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)


import numpy as np
import os
import sys
import time
import pandas as pd 
#from tqdm._tqdm_notebook import tqdm_notebook
import pickle

cutoff = 10.0

# from keras.wrappers.scikit_learn import KerasClassifier
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#import logging


/home/aj/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aj/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aj/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aj/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
INPUT_PATH = "/home/aj/eurusd"
TIMESTEPS = 100
BATCHSIZE = 100
NUMRECORDS = 10398
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [3]:
eurusddata = pd.read_csv(os.path.join(INPUT_PATH, "EURUSDDAILY.csv"), engine='python')
print(eurusddata.shape)
print(eurusddata.columns)

#display(eurusddata.head(5))
#display(eurusddata.tail(1451))



dim_0 = eurusddata.shape[0] - TIMESTEPS 
dim_1 = eurusddata.shape[1] -2
print(dim_0)
print(dim_1)
dim_1 = 1

(10398, 6)
Index(['Date', 'Price', 'Open', 'High', 'Low', 'Change %'], dtype='object')
10298
4


In [4]:
#ffg
pricearray2 = np.zeros((10398,2))
pricearray = np.zeros((10398,1))

#gg
eurusddata1 = eurusddata.iloc[::-1]
display(eurusddata1.head(1451))
#display(eurusddata1.tail(5))


# IMPORTANT: Data flat from record# 0 till record# 1450



,Date,Price,Open,High,Low,Change %
10397,8-Aug-80,1.4599,1.4599,1.4599,1.4599,-0.08%
10396,11-Aug-80,1.4592,1.4592,1.4592,1.4592,-0.05%
10395,12-Aug-80,1.4607,1.4607,1.4607,1.4607,0.10%
10394,13-Aug-80,1.4620,1.4620,1.4620,1.4620,0.09%
10393,14-Aug-80,1.4571,1.4571,1.4571,1.4571,-0.34%
...,...,...,...,...,...,...
8951,30-Apr-86,0.9938,0.9938,0.9938,0.9938,0.09%
8950,1-May-86,0.9793,0.9793,0.9793,0.9793,-1.46%
8949,2-May-86,0.9790,0.9790,0.9790,0.9790,-0.03%
8948,6-May-86,0.9747,0.9747,0.9747,0.9747,-0.44%


        
for i in range(dim_0+TIMESTEPS):
            
            #print(TIMESTEPS+j+1)
        pricearray[i] = eurusddata1.iloc[0+i, 3:4]

In [5]:
for i in range(dim_0+TIMESTEPS):
            
            #print(TIMESTEPS+j+1)
        pricearray2[i] = eurusddata1.iloc[0+i, 3:5]

In [6]:
print(pricearray2[1449])

print(pricearray2[1451])

#pricearray = pricearray2[0] - pricearray2[1]

yy = pricearray2[:, 0] 

yyy = pricearray2[:, 1] 


pricearray  = pricearray2[:, 0]  - pricearray2[:, 1] 

[0.9747 0.9747]
[0.9851 0.9804]


In [7]:
pricearray.shape

(10398,)

In [8]:
yyy.shape

(10398,)

In [9]:
print(pricearray[1449])


print(pricearray[1451])
offsetmin = np.amin(pricearray)
maxinrange = np.amax(pricearray)
print(offsetmin)
print(maxinrange)

0.0
0.0046999999999999265
0.0
0.06879999999999997


In [10]:
print(pricearray[1449])
print(pricearray[1451])
print(pricearray[2451])
#pricearray = pricearray/(maxinrange + 0.0001)
print(pricearray[1449])
print(pricearray[1451])
print(pricearray[2451])

0.0
0.0046999999999999265
0.006399999999999961
0.0
0.0046999999999999265
0.006399999999999961


In [11]:
offsetmin = np.amin(pricearray)
maxinrange = np.amax(pricearray)
print(offsetmin)
print(maxinrange)

0.0
0.06879999999999997


In [12]:
#for i in range(len(pricearray)):
   # if pricearray[i] == 0.0:
     #   print(i)

In [13]:
#fsdfsa

In [14]:
x = np.zeros((dim_0-1450, 1, TIMESTEPS ))
y = np.zeros((dim_0-1450, dim_1))

In [15]:
x.shape


(8848, 1, 100)

In [16]:
y.shape

(8848, 1)

In [17]:

#pricearray = pricearray.reshape(10398, 1)

print(pricearray[1449])
print(pricearray[1451])
print(pricearray[2451])


0.0
0.0046999999999999265
0.006399999999999961


In [18]:
'''
arr = eurusddata1.to_numpy()
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [120, 80]
x = np.arange(10398)

y1 = arr[:, 1]
y2 = arr[:, 2]
y3 = arr[:, 3]
y4 = arr[:, 4]
print(y1)
print(y2)
print(y3)
print(y4)


#plt.plot(x, y1, label="John")

#plt.plot(x, y2, label="David")

#plt.plot(x, y3, label="Tom")

plt.plot(x, y3 - y4, label="Tomato")

pricearray
'''


eurusddata1 = eurusddata.iloc[::-1]
#display(eurusddata1.head(61))
#display(eurusddata1.tail(5))

for i in range(dim_0-1450):
            
            #print(TIMESTEPS+j+1)
            s = pricearray[1450+i:1450+TIMESTEPS+i]
            x[i] = s.reshape(1,TIMESTEPS)
            y[i] = pricearray[1450+TIMESTEPS+i]

In [19]:


print(x[0])
print(y[0])

print(x[1])
print(y[1])

print(x[10296-1450])
print(y[10296-1450])

print(x[10297-1450])
print(y[10297-1450])


[[0.0062 0.0047 0.0052 0.0152 0.0066 0.0075 0.0062 0.0029 0.0161 0.0052
  0.0072 0.009  0.0009 0.009  0.0071 0.0067 0.0066 0.0099 0.0156 0.008
  0.0095 0.0057 0.0071 0.0147 0.0052 0.0066 0.0062 0.0052 0.0076 0.0109
  0.0047 0.0028 0.0052 0.0061 0.0071 0.0023 0.0062 0.0052 0.0057 0.0052
  0.0038 0.0019 0.0075 0.0071 0.0109 0.0042 0.0024 0.0042 0.0062 0.0047
  0.0043 0.0023 0.0085 0.0086 0.0109 0.0062 0.0067 0.0076 0.0071 0.0071
  0.0042 0.0056 0.0066 0.0071 0.0047 0.0033 0.0066 0.0047 0.0062 0.0033
  0.0024 0.0029 0.0057 0.0109 0.0053 0.0052 0.0043 0.0024 0.0033 0.0066
  0.0062 0.0109 0.0034 0.0033 0.0075 0.0052 0.0086 0.0095 0.0075 0.0061
  0.0122 0.0194 0.0085 0.0053 0.009  0.0095 0.0056 0.009  0.0095 0.009 ]]
[0.0028]
[[0.0047 0.0052 0.0152 0.0066 0.0075 0.0062 0.0029 0.0161 0.0052 0.0072
  0.009  0.0009 0.009  0.0071 0.0067 0.0066 0.0099 0.0156 0.008  0.0095
  0.0057 0.0071 0.0147 0.0052 0.0066 0.0062 0.0052 0.0076 0.0109 0.0047
  0.0028 0.0052 0.0061 0.0071 0.0023 0.0062 0.0052 0.0

In [20]:
'''
x1 = x[:,:,1]
y1 = y[:,:,1]
print(x1)
print(y1)
'''

'\nx1 = x[:,:,1]\ny1 = y[:,:,1]\nprint(x1)\nprint(y1)\n'

In [21]:
'''
x = x1 - x0
y = y1 - y0
for i in range(1):
           print(x[i])
           print(y[i])

j=1

print(x[3015])
print(y[3015])
x =  x[3015:,:]
y =  y[3015:,:]
'''


'\nx = x1 - x0\ny = y1 - y0\nfor i in range(1):\n           print(x[i])\n           print(y[i])\n\nj=1\n\nprint(x[3015])\nprint(y[3015])\nx =  x[3015:,:]\ny =  y[3015:,:]\n'

In [22]:
x.shape

(8848, 1, 100)

In [23]:
y.shape

(8848, 1)

In [24]:
offsetmin = np.amin(y)
maxinrange = np.amax(y)
print(offsetmin)
print(maxinrange)

0.0
0.06879999999999997


In [25]:
for i in range(5):
    print(x[i])
    print(y[i])

[[0.0062 0.0047 0.0052 0.0152 0.0066 0.0075 0.0062 0.0029 0.0161 0.0052
  0.0072 0.009  0.0009 0.009  0.0071 0.0067 0.0066 0.0099 0.0156 0.008
  0.0095 0.0057 0.0071 0.0147 0.0052 0.0066 0.0062 0.0052 0.0076 0.0109
  0.0047 0.0028 0.0052 0.0061 0.0071 0.0023 0.0062 0.0052 0.0057 0.0052
  0.0038 0.0019 0.0075 0.0071 0.0109 0.0042 0.0024 0.0042 0.0062 0.0047
  0.0043 0.0023 0.0085 0.0086 0.0109 0.0062 0.0067 0.0076 0.0071 0.0071
  0.0042 0.0056 0.0066 0.0071 0.0047 0.0033 0.0066 0.0047 0.0062 0.0033
  0.0024 0.0029 0.0057 0.0109 0.0053 0.0052 0.0043 0.0024 0.0033 0.0066
  0.0062 0.0109 0.0034 0.0033 0.0075 0.0052 0.0086 0.0095 0.0075 0.0061
  0.0122 0.0194 0.0085 0.0053 0.009  0.0095 0.0056 0.009  0.0095 0.009 ]]
[0.0028]
[[0.0047 0.0052 0.0152 0.0066 0.0075 0.0062 0.0029 0.0161 0.0052 0.0072
  0.009  0.0009 0.009  0.0071 0.0067 0.0066 0.0099 0.0156 0.008  0.0095
  0.0057 0.0071 0.0147 0.0052 0.0066 0.0062 0.0052 0.0076 0.0109 0.0047
  0.0028 0.0052 0.0061 0.0071 0.0023 0.0062 0.0052 0.0

In [26]:
arr_rand = np.random.rand(x.shape[0])
split = arr_rand < np.percentile(arr_rand, 90)

x_train = x[split]
y_train = y[split]
x_testeval = x[~split]
y_testeval = y[~split]

print(x_train.shape)
print(y_train.shape)
print(x_testeval.shape) 
print(y_testeval.shape)

arr_rand1 = np.random.rand(x_testeval.shape[0])
split1 = arr_rand1 < np.percentile(arr_rand1, 90)

x_test = x_testeval[split1]
y_test = y_testeval[split1]
x_eval = x_testeval[~split1]
y_eval = y_testeval[~split1]

print(x_test.shape) 
print(y_test.shape)
print(x_eval.shape) 
print(y_eval.shape)



(7963, 1, 100)
(7963, 1)
(885, 1, 100)
(885, 1)
(796, 1, 100)
(796, 1)
(89, 1, 100)
(89, 1)


In [27]:
for i in range(5):
    print(x_test[i])
    print(y_test[i])

[[0.0072 0.009  0.0009 0.009  0.0071 0.0067 0.0066 0.0099 0.0156 0.008
  0.0095 0.0057 0.0071 0.0147 0.0052 0.0066 0.0062 0.0052 0.0076 0.0109
  0.0047 0.0028 0.0052 0.0061 0.0071 0.0023 0.0062 0.0052 0.0057 0.0052
  0.0038 0.0019 0.0075 0.0071 0.0109 0.0042 0.0024 0.0042 0.0062 0.0047
  0.0043 0.0023 0.0085 0.0086 0.0109 0.0062 0.0067 0.0076 0.0071 0.0071
  0.0042 0.0056 0.0066 0.0071 0.0047 0.0033 0.0066 0.0047 0.0062 0.0033
  0.0024 0.0029 0.0057 0.0109 0.0053 0.0052 0.0043 0.0024 0.0033 0.0066
  0.0062 0.0109 0.0034 0.0033 0.0075 0.0052 0.0086 0.0095 0.0075 0.0061
  0.0122 0.0194 0.0085 0.0053 0.009  0.0095 0.0056 0.009  0.0095 0.009
  0.0028 0.0047 0.0048 0.0085 0.0033 0.0066 0.0099 0.0062 0.0133 0.0042]]
[0.0057]
[[0.0066 0.0099 0.0156 0.008  0.0095 0.0057 0.0071 0.0147 0.0052 0.0066
  0.0062 0.0052 0.0076 0.0109 0.0047 0.0028 0.0052 0.0061 0.0071 0.0023
  0.0062 0.0052 0.0057 0.0052 0.0038 0.0019 0.0075 0.0071 0.0109 0.0042
  0.0024 0.0042 0.0062 0.0047 0.0043 0.0023 0.0085 0.00

In [28]:
#x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1])) # Reshaping done for LSTM as it needs 3D input
#x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))
#x_eval = x_eval.reshape((x_eval.shape[0], 1, x_eval.shape[1]))

In [29]:
#seerwe

In [30]:
def totaltestpredictionerror(y_test, y_testresult):
    ytestdiff = y_test - y_testresult
    ytestdiffabs = np.absolute(ytestdiff)
    sumoftesterrors = np.sum(ytestdiffabs) 
    y_test = None
    y_testresult = None
    ytestdiff = None
    ytestdiffabs =  None
    
    return sumoftesterrors

In [31]:
def totalevalpredictionerror(y_eval, y_evalresult):
    yevaldiff = y_eval - y_evalresult
    yevaldiffabs = np.absolute(yevaldiff)
    sumofevalerrors = np.sum(yevaldiffabs)
    y_eval = None
    y_evaltresult = None
    yevaldiff = None
    ytestdiffabs =  None
    
    return sumofevalerrors

In [32]:
def totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult):
    ytestdiff = y_test - y_testresult
    yevaldiff = y_eval - y_evalresult
    ytestdiffabs = np.absolute(ytestdiff)
    yevaldiffabs = np.absolute(yevaldiff)
    sumoferrors = np.sum(ytestdiffabs) + np.sum(yevaldiffabs)
    y_eval = None
    y_evaltresult = None
    y_test = None
    y_testresult = None
    ytestdiff = None
    yevaldiff = None
    ytestdiffabs =  None
    
    return sumoferrors

In [33]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape) 
print(y_test.shape)
print(x_eval.shape) 
print(y_eval.shape)

(7963, 1, 100)
(7963, 1)
(796, 1, 100)
(796, 1)
(89, 1, 100)
(89, 1)


In [34]:
unitsize = 800

regressor = Sequential()

regressor.add(LSTM(units = unitsize, return_sequences = True, input_shape = (x_train.shape[1], x_train.shape[2])))

regressor.add(LSTM(units = unitsize, return_sequences = True))

regressor.add(LSTM(units = unitsize, return_sequences = True))

regressor.add(LSTM(units = unitsize, return_sequences = True))

regressor.add(LSTM(units = unitsize))

regressor.add(Dense(units = 1))

opt = tkeras.optimizers.Adam(learning_rate=0.0006)
regressor.compile(loss='mean_squared_error', optimizer=opt)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [35]:

with tf.device('gpu'):
    lowfilenumber = 0
    lowscore = 100
    for i in range(60000):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1024, verbose=0)
        #regressor.save("EUR24H800Xrangesaving.h5")
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)
        

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        #print(i, ": Total -- -------------", ytotaldiff)
        
        if lowscore > ytotaldiff:
            lowscore = ytotaldiff
            lowfilenumber = i
            print("lowfilenumber-", lowfilenumber,"                          LOWSCORE - ", lowscore)
            regressor.save("EUR24H800Xrangereal.h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
        
        #cutoff = 3.69
        #if ytotaldiff < cutoff:
            #print(i+0)
            #regressor.save("EUR24H800Xrange"+str(i+0)+".h5")
            #print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
            
       
            
       

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
lowfilenumber- 0                           LOWSCORE -  3.399005474962296
  Test --  3.0141017098546046   Eval --  0.3849037651076914   Total -- ------------- 3.399005474962296
lowfilenumber- 2                           LOWSCORE -  3.372458536291124
  Test --  2.9890395611822616   Eval --  0.38341897510886236   Total -- ------------- 3.372458536291124
lowfilenumber- 77                           LOWSCORE -  3.3688712978005424
  Test --  2.9856876166835438   Eval --  0.38318368111699863   Total -- ------------- 3.3688712978005424
lowfilenumber- 154                           LOWSCORE -  3.3662996160611525
  Test --  2.9832960137590776   Eval --  0.38300360230207486   Total -- ------------- 3.3662996160611525
lowfilenumber- 241                           LOWSCORE -  3.3647196415081635
  Test --  2.9821402658075105   Eval --  0.38257937570065303   Total -- ------------- 3.3647196415081635
lowfilenumb

KeyboardInterrupt: 

In [ ]:
bfghfgj
from tensorflow import keras
from tensorflow.keras import layers

regressor = load_model("EURUSD24H800X59228.h5")

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.005,
    decay_steps=1000,
    decay_rate=0.75,
    staircase=True)




In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.002)
regressor.compile(loss='mean_squared_error', optimizer=opt)

In [ ]:
gtc

In [ ]:
with tf.device('gpu'):
    for i in range(75000):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028, verbose=0)
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        #print("  Total -- -------------", ytotaldiff)
        cutoff = 3.51
        if ytotaldiff < cutoff:
            print(i+60)
            regressor.save("EURUSD24H"+str(i+60)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
            
       

In [ ]:

with tf.device('gpu'):
    for i in range(600):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        
        if ytotaldiff < 5.2:
            print(i)
            regressor.save("EURUSD24H"+str(i))+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
       

In [ ]:

with tf.device('gpu'):
    for i in range(600):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        
        if ytotaldiff < 4.7:
            print(i+600)
            regressor.save("EURUSD24H"+str(i+600)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
       

In [ ]:

with tf.device('gpu'):
    for i in range(600):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        
        if ytotaldiff < 4.5:
            print(i+1200)
            regressor.save("EURUSD24H"+str(i+1200)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
       

In [ ]:

with tf.device('gpu'):
    for i in range(1200):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        
        if ytotaldiff < 4.4:
            print(i+1800)
            regressor.save("EURUSD24H"+str(i+1800)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
       

In [ ]:
with tf.device('gpu'):
    for i in range(3000):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        
        if ytotaldiff < 4.08:
            print(i+3000)
            regressor.save("EURUSD24H"+str(i+3000)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
       

In [ ]:
with tf.device('gpu'):
    for i in range(6000):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        
        if ytotaldiff < 3.5348:
            print(i+18000)
            regressor.save("EURUSD24H"+str(i+18000)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
       

In [ ]:

with tf.device('gpu'):
    for i in range(20):
        
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        
        y_testresult = regressor.predict(x_test)
        y_evalresult = regressor.predict(x_eval)

        ytestdiff = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)
        #print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
        
        
        cutoff1 = 3.4281
        if ytotaldiff < cutoff1:
            print(" cutoff applied  -- " , cutoff1)
            print(i+60000)
            regressor.save("EURUSD24H"+str(i+60000)+".h5")
            print("  Test -- ", ytestdiff, "  Eval -- ", yevaldiff, "  Total -- -------------", ytotaldiff)
            cutoff1 = ytotaldiff
            print("new globval cutoff -- " , cutoff1)
       

In [ ]:
with tf.device('gpu'):
    for i in range(1):
        print(i)
        history = regressor.fit(x_train, y_train, epochs = 1, batch_size = 1028)
        regressor.save("EURUSDDAILYCLOSEM7.h5")

In [ ]:
y_testresult = regressor.predict(x_test)
y_evalresult = regressor.predict(x_eval)

In [ ]:
for i in range(5):
    print(y_testresult[i])
    print(y_test[i]) 

In [ ]:
for i in range(5):
    print(y_testresult[i])
    print(y_test[i]) 

In [ ]:
plot.plot(history.history['loss'], label='MAE (training data)')
#plot.plot(history.history['val_loss'], label='MAE (validation data)')

plot.show()

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [120, 80]
x = np.arange(len(y_testresult))
y=y_testresult.reshape(927)
y1=y_test.reshape(927)

yyy = y_testresult - y_test
ydif = yyy.reshape(927)

#plt.plot(x, y, label="John")

#plt.plot(x, y1, label="David")

plt.plot(x, y_testresult - y_test, label="Tom")



In [ ]:
good till here


In [ ]:
scrap
def ResNetFX(input_shape = (1,100)):
    """
    Arguments:
    input_shape -- shape 

    Returns:
    model -- a Model() instance in Keras
    """
    """
    size = 3200
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    X = Dense(size,  activation='relu')(X_input)
    X = Dense(size,  activation='relu')(X)
    X = Dense(size,  activation='relu')(X)
    X = Dense(size,  activation='relu')(X)
    X = Dense(size,  activation='relu')(X)
    X = Dense(size,  activation='relu')(X)
    X = Dense(size,  activation='relu')(X)
    X = Dense(size,  activation='relu')(X)
    X = Dense(1,  activation='linear')(X)
    """
    
    X_input = Input(input_shape)
    X = LSTM(100, input_shape=(train_X.shape[1], train_X.shape[2]))
    X = Dense(1,  activation='linear')(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNetFX')

    return model


In [ ]:
scrapmodel = ResNetFX(input_shape = (100))

'''
model.compile(optimizer='adam', loss='mse', metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), 
    tf.keras.metrics.Recall(), tf.keras.metrics.TruePositives(), tf.keras.metrics.TrueNegatives(),
    tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives(), tf.keras.metrics.SensitivityAtSpecificity(),
    tf.keras.metrics.SpecificityAtSensitivity()])
'''
model.compile(optimizer='adam', loss = 'mean_squared_error', metrics=['mse'])

#model.fit(x_train, y_train, epochs = 10, batch_size = 8)



In [ ]:
scrap
gc.collect()
offset = 0

ytestdiff = np.zeros(600)
yevaldiff = np.zeros(600)
ytotaldiff = np.zeros(600)

#model = load_model("EURUSDDAILY"+str(offset-1)+".h5")
#model.compile(optimizer='adam', loss='mean_squared_error', 
              #metrics=[tf.keras.metrics.MeanAbsoluteError()])

with tf.device('gpu'):
    for i in range(600):
        history = model.fit(x_train, y_train, epochs = 1, batch_size = 64)
        model.save("EURUSD24H"+str(i+offset)+".h5")
        y_testresult = model.predict(x_test)
        y_evalresult = model.predict(x_eval)

        ytestdiff[i] = totaltestpredictionerror(y_test, y_testresult)
        yevaldiff[i] = totalevalpredictionerror(y_eval, y_evalresult)
        ytotaldiff[i] = totalpredictionerror(y_test, y_testresult, y_eval, y_evalresult)

        print("Iter#--", i, )
        print("  Test -- ", ytestdiff[i], "  Eval -- ", yevaldiff[i], "  Total -- ", ytotaldiff[i])


        gc.collect()
    #hist_df = pd.DataFrame(history.history) 
    # or save to csv: 
    #hist_csv_file = "EURUSDDailyhistory"+str(i+offset)+".csv"
    #with open(hist_csv_file, mode='w') as f:
        #hist_df.to_csv(f)

In [ ]:
scrap
for i in range(50):
    print(y_test[i])
    print(y_testresult[i])